This notebook is a prototype of the rTMS Bayesian Optimization algorithm and will reflect the final step of pipeline after tetrahedral projection and FEM surface construction/registration

In [1]:
import os
import numpy as np
from fieldopt import geolib
from fieldopt.objective import FieldFunc
from timeit import default_timer as timer

In [2]:
#MOE package loading (should wrap!)
from moe.optimal_learning.python.cpp_wrappers.domain import TensorProductDomain as cTensorProductDomain
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import ExpectedImprovement
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import multistart_expected_improvement_optimization as meio
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint
from moe.optimal_learning.python.cpp_wrappers.log_likelihood_mcmc import GaussianProcessLogLikelihoodMCMC
from moe.optimal_learning.python.default_priors import DefaultPrior
from moe.optimal_learning.python.python_version.optimization import GradientDescentOptimizer, GradientDescentParameters
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentOptimizer as cGDOpt
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentParameters as cGDParams

In [3]:
#Specify inputs
proj_dir =  '/rtms_optimize'
mesh_file = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
coil_file = os.path.join(proj_dir,'resources','coils','Magstim_70mm_Fig8.nii.gz')
tet_file = os.path.join(proj_dir,'output','tetra_parcels')
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
testing_dir = '/tmp/'#os.path.join(proj_dir,'testing','tmp')

In [4]:
#Step 1: Load in files containing relevant information about search domain
C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

#Load in element mask
p_map = np.load(os.path.join(proj_dir,'output','tetra_parcels.npy'))[:,2]

In [5]:
#Step 2: Specify the search domain
search_domain = TensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), #X coord on quadratic surface
    ClosedInterval(b[1,0],b[1,1]), #Y coord on quadratic surface
    ClosedInterval(0,180) #Rotational angle
])

c_search_domain = cTensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), 
    ClosedInterval(b[1,0],b[1,1]),
    ClosedInterval(0,180)
])

In [6]:
#Make objective function
ncpus = 15
f = FieldFunc(mesh_file=mesh_file, quad_surf_consts=C,
              surf_to_mesh_matrix=iR, tet_weights=p_map,
              field_dir=testing_dir, coil=coil_file, cpus=ncpus)

In [17]:
#Generate historical data 
hist_pts = int(ncpus * 1.5)
init_pts = search_domain.generate_uniform_random_points_in_domain(hist_pts)
start = timer()
observations = f.evaluate(init_pts)
stop = timer()
print(stop - start)

[ simnibs ]INFO: Could not find the dti tensor file in the standard location: /rtms_optimize/data/simnibs_output/d2c_sub-CMH090/dti_results_T1space/DTI_conf_tensor.nii.gz
[ simnibs ]INFO: Head Mesh: /rtms_optimize/data/simnibs_output/sub-CMH090.msh
[ simnibs ]INFO: Subject Path: /rtms_optimize/data/simnibs_output/m2m_sub-CMH090
[ simnibs ]INFO: Simulation Folder: /tmp/tmp2934Vq
[ simnibs ]INFO: Running simulations in the directory: /tmp/tmp2934Vq
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Using coil: /rtms_optimize/resources/coils/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities


0
1


[ simnibs ]INFO: Running TMS Position 0


2


[ simnibs ]INFO: Coil Position Matrix: [[ 1.58074002e-02 -5.86537061e-01  8.09768116e-01 -6.03975757e+01]
 [ 2.90441884e-01 -7.72255509e-01 -5.65035345e-01  6.84103810e+01]
 [-9.56762059e-01 -2.44122317e-01 -1.58147577e-01  1.39553722e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
dIdt: 1000000.0

[ simnibs ]INFO: Running TMS Position 1


...

[ simnibs ]INFO: Info    : 126 KSP Residual norm 5.090902991746e-10
[ simnibs ]INFO: Info    : 121 KSP Residual norm 1.652923068478e-09
[ simnibs ]INFO: Info    : 125 KSP Residual norm 4.979328378207e-10
[ simnibs ]INFO: Info    : 127 KSP Residual norm 4.367329671895e-10
[ simnibs ]INFO: Info    : 118 KSP Residual norm 2.670694636607e-09
[ simnibs ]INFO: Info    : 109 KSP Residual norm 2.042011928796e-08
[ simnibs ]INFO: Info    : 126 KSP Residual norm 3.841920358123e-10
[ simnibs ]INFO: Info    : 114 KSP Residual norm 5.386608198992e-09
[ simnibs ]INFO: Info    : 114 KSP Residual norm 6.203511003004e-09
[ simnibs ]INFO: Info    : 129 KSP Residual norm 1.816693156057e-10
[ simnibs ]INFO: Info    : 113 KSP Residual norm 3.648906559694e-09
[ simnibs ]INFO: Info    : 109 KSP Residual norm 1.953438651043e-08
[ simnibs ]INFO: Info    : 125 KSP Residual norm 5.700101307633e-10
[ simnibs ]INFO: Info    : 109 KSP Residual norm 1.635264447273e-08
[ simnibs ]INFO: Info    : 115 KSP Residual norm

[ simnibs ]INFO: Info    : 133 KSP Residual norm 1.111519730107e-10
[ simnibs ]INFO: Info    : 116 KSP Residual norm 4.475768157329e-09
[ simnibs ]INFO: Info    : 135 KSP Residual norm 6.162188752566e-11
[ simnibs ]INFO: Info    : 126 KSP Residual norm 4.440483519646e-10
[ simnibs ]INFO: Info    : 122 KSP Residual norm 1.008593928941e-09
[ simnibs ]INFO: Info    : 134 KSP Residual norm 6.992219890641e-11
[ simnibs ]INFO: Info    : 121 KSP Residual norm 7.572920639374e-10
[ simnibs ]INFO: Info    : 133 KSP Residual norm 1.096569190405e-10
[ simnibs ]INFO: Info    : 122 KSP Residual norm 1.362503260655e-09
[ simnibs ]INFO: Info    : 137 KSP Residual norm 4.484950373648e-11
[ simnibs ]INFO: Info    : 117 KSP Residual norm 2.636471540327e-09
[ simnibs ]INFO: Info    : 123 KSP Residual norm 9.034397317791e-10
[ simnibs ]INFO: Info    : 135 KSP Residual norm 6.112419737496e-11
[ simnibs ]INFO: Info    : 130 KSP Residual norm 3.212393630581e-10
[ simnibs ]INFO: Info    : 117 KSP Residual norm

[ simnibs ]INFO: Info    : 128 KSP Residual norm 1.683229145946e-10
[ simnibs ]INFO: Info    : 129 KSP Residual norm 2.749611862166e-10
[ simnibs ]INFO: Info    : 124 KSP Residual norm 6.586730565048e-10
[ simnibs ]INFO: Info    : 129 KSP Residual norm 3.256657388799e-10
[ simnibs ]INFO: Execution finished
[ simnibs ]INFO: Info    : 130 KSP Residual norm 1.518365967227e-10
[ simnibs ]INFO: Info    : 124 KSP Residual norm 1.065604804241e-09
[ simnibs ]INFO: Info    : 124 KSP Residual norm 1.067892425046e-09
[ simnibs ]INFO: Info    : 134 KSP Residual norm 5.944687829926e-11
[ simnibs ]INFO: Info    : 138 KSP Residual norm 3.680332575645e-11
[ simnibs ]INFO: Info    : 138 iterations
[ simnibs ]INFO: Info    : SaveSolution[QS_dAdt]
[ simnibs ]INFO: Info    : (Wall = 285.027s, CPU = 283.718s, Mem = 4244.28Mb)
[ simnibs ]INFO: E n d   P r o c e s s i n g
[ simnibs ]INFO: Info    : 129 KSP Residual norm 1.407612982096e-10
[ simnibs ]INFO: Info    : 130 KSP Residual norm 2.086289963533e-10
[ 

[ simnibs ]INFO: Info    : 138 iterations
[ simnibs ]INFO: Info    : SaveSolution[QS_dAdt]
[ simnibs ]INFO: Info    : (Wall = 294.386s, CPU = 293.611s, Mem = 4244.26Mb)
[ simnibs ]INFO: E n d   P r o c e s s i n g
[ simnibs ]INFO: Info    : 138 KSP Residual norm 3.538140792990e-11
[ simnibs ]INFO: Info    : 138 iterations
[ simnibs ]INFO: Info    : SaveSolution[QS_dAdt]
[ simnibs ]INFO: Info    : Stopped (Fri Apr 26 16:14:58 2019, Wall = 292.944s, CPU = 291.417s, Mem = 4244.28Mb)
[ simnibs ]INFO: Info    : (Wall = 294.174s, CPU = 293.022s, Mem = 4244.15Mb)
[ simnibs ]INFO: E n d   P r o c e s s i n g
[ simnibs ]INFO: Info    : Stopped (Fri Apr 26 16:14:58 2019, Wall = 294.839s, CPU = 293.796s, Mem = 4244.26Mb)
[ simnibs ]INFO: Execution finished
[ simnibs ]INFO: Info    : Stopped (Fri Apr 26 16:14:58 2019, Wall = 294.642s, CPU = 293.233s, Mem = 4244.15Mb)
[ simnibs ]INFO: Execution finished
[ simnibs ]INFO: Execution finished
[ simnibs ]INFO: Execution finished
[ simnibs ]INFO: Executi

542.239037991


In [23]:
observations = -observations

In [25]:
hist_data = HistoricalData(dim = 3, num_derivatives= 0)
hist_data.append_sample_points([SamplePoint(inp,o,0.0) 
                                for o,inp in 
                                zip(observations,init_pts)])


In [26]:
#Set up model specifications
prior = DefaultPrior(n_dims = 3 + 2, num_noise=1)
gp_ll = GaussianProcessLogLikelihoodMCMC(historical_data=hist_data,
                                         derivatives=[], prior=prior,
                                         chain_length=1000, burnin_steps=2000,
                                         n_hypers=2**4, noisy=False)
gp_ll.train()

In [27]:
#Set up optimizer
sgd_params = cGDParams(num_multistarts=200, max_num_steps=50,
                      max_num_restarts=2, num_steps_averaged=4,
                      gamma=0.7, pre_mult=1.0, max_relative_change=0.5,
                      tolerance=1.0e-10)

In [44]:
#Set up optimization parametes
from IPython.display import clear_output, display
num_iters = 50
num_samples = int(ncpus*1.3)

In [45]:
def gen_sample_from_qei(gp,search_domain,sgd_params,num_samples, num_mc=1e4, lhc_iter=2e4):
    
    qEI = ExpectedImprovement(gaussian_process=gp, num_mc_iterations=int(num_mc))
    optimizer = cGDOpt(search_domain, qEI, sgd_params, int(lhc_iter))
    points_to_sample = meio(optimizer, None, num_samples, use_gpu=False, which_gpu=0,
                      max_num_threads=8)
    qEI.set_current_point(points_to_sample[0])
    
    return points_to_sample, qEI.compute_expected_improvement()

In [48]:
best_point_history = []
for i in np.arange(0,num_iters):
    
    #Optimize qEI and pick samples
    points_to_sample, ei = gen_sample_from_qei(gp_ll.models[0],
                                               c_search_domain, sgd_params=sgd_params,
                                               num_samples=num_samples, num_mc=2**10)
    
    #Collect observations
    sampled_points = -f.evaluate(points_to_sample)
    evidence = [SamplePoint(c,v,0.0) for c,v in zip(points_to_sample, sampled_points)]
    
    #Update model
    gp_ll.add_sampled_points(evidence)
    gp_ll.train()
    
    #Pull model and pull values
    gp = gp_ll.models[0]
    min_point = np.argmin(gp._points_sampled_value)
    min_val = np.min(gp._points_sampled_value)
    best_coord = gp.get_historical_data_copy().points_sampled[min_point]
    
    clear_output()
    print('Recommended Points:')
    print(points_to_sample)
    print('Expected Improvement: {}'.format(ei))
    print('Current Best:')
    print('f(x*)=',min_val)
    print('Coord:', best_coord)
    
    best_point_history.append(min_val)

Recommended Points:
[[-3.41717108e+00 -1.38474796e-01  1.36363716e+02]
 [ 6.35021787e+00  1.70156497e+01  9.37690190e+01]
 [ 2.90076500e+01  2.78538462e+01  8.77297149e+00]
 [ 2.80575849e+01 -7.52749833e+00  3.60256187e+01]
 [ 5.91964109e+00  3.73052597e+00  1.18923099e+02]
 [-8.63318924e+00  1.11079295e+01  4.88326049e+01]
 [ 1.55585938e+01 -1.02555772e+00  1.62063422e+02]
 [ 8.22779890e-01 -8.21376821e-01  7.03603071e+00]
 [ 1.65091303e+00 -8.58093727e+00  2.64030644e+01]
 [ 2.57771091e+01 -5.93440475e+00  1.04166824e+02]
 [ 4.44502440e+00  3.48400747e+01  1.13575240e+02]
 [-2.87805056e+00  3.39862381e+00  1.72412585e+02]
 [ 1.72004122e+01 -3.88434616e+00  1.66441535e+02]
 [ 1.69436039e+01  4.57247399e+00  8.79119903e+01]
 [-4.38331987e+00  8.88195348e+00  1.43422487e+02]
 [ 1.72165658e+01 -8.43848898e+00  1.64618252e+01]
 [ 7.60091948e+00  1.98176568e+01  1.03007641e+02]
 [ 1.21526358e+00  2.19944907e+00  5.23628059e+01]
 [-5.04151204e+00  1.41269781e+01  6.41187286e+00]]
Expected I